# Startlink Performance Monitoring with AI
## Author: Fatih E. NAR

In [ ]:
#%pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import joblib
from accelerate import Accelerator
from torch.utils.data import DataLoader, TensorDataset

In [3]:
class StarlinkTransformer(nn.Module):
    def __init__(self, input_dim, num_heads=4, dim_feedforward=128, dropout=0.1):
        super().__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=input_dim,
                nhead=num_heads,
                dim_feedforward=dim_feedforward,
                dropout=dropout
            ),
            num_layers=2
        )
        self.fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        x = self.transformer(x)
        x = torch.mean(x, dim=1)
        return self.fc(x)

In [4]:
# Data preparation
df = pd.read_csv('data/starlink_data.csv')

label_encoders = {}
for cat_feat in ['season', 'weather']:
    label_encoders[cat_feat] = LabelEncoder()
    df[f'{cat_feat}_encoded'] = label_encoders[cat_feat].fit_transform(df[cat_feat])

features = ['latitude', 'longitude', 'elevation_m', 'visible_satellites', 
           'serving_satellites', 'signal_loss_db', 'season_encoded', 'weather_encoded']

X = df[features].values
y = df['qoe_score'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to tensors and create dataloaders
X_tensor = torch.FloatTensor(X_scaled)
y_tensor = torch.FloatTensor(y).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128)

In [5]:
# Initialize accelerator
accelerator = Accelerator()

# Model setup
model = StarlinkTransformer(input_dim=X.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Prepare for accelerated training
model, optimizer, train_loader, test_loader = accelerator.prepare(
    model, optimizer, train_loader, test_loader
)

# Training loop
num_epochs = 200

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X.unsqueeze(1))
        loss = criterion(outputs, batch_y)
        accelerator.backward(loss)
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for test_X, test_y in test_loader:
                y_pred = model(test_X.unsqueeze(1))
                test_loss += criterion(y_pred, test_y).item()
        print(f'Epoch {epoch+1}, Test Loss: {test_loss/len(test_loader):.4f}')

/opt/app-root/lib64/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 10, Test Loss: 0.2680
Epoch 20, Test Loss: 0.1621
Epoch 30, Test Loss: 0.1255
Epoch 40, Test Loss: 0.1554
Epoch 50, Test Loss: 0.0680
Epoch 60, Test Loss: 0.0669
Epoch 70, Test Loss: 0.0669
Epoch 80, Test Loss: 0.0574
Epoch 90, Test Loss: 0.1041
Epoch 100, Test Loss: 0.0582
Epoch 110, Test Loss: 0.0644
Epoch 120, Test Loss: 0.1123
Epoch 130, Test Loss: 0.0619
Epoch 140, Test Loss: 0.0601
Epoch 150, Test Loss: 0.1175
Epoch 160, Test Loss: 0.0641
Epoch 170, Test Loss: 0.0534
Epoch 180, Test Loss: 0.0643
Epoch 190, Test Loss: 0.0623
Epoch 200, Test Loss: 0.0603


In [6]:
# Save model and preprocessing
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
torch.save(unwrapped_model.state_dict(), 'models/starlink_transformer.pth')
joblib.dump(scaler, 'models/scaler.joblib')
joblib.dump(label_encoders, 'models/label_encoders.joblib')

['models/label_encoders.joblib']

In [7]:
# Final evaluation
model.eval()
total_loss = 0
y_true = []
y_pred_list = []

with torch.no_grad():
    for test_X, test_y in test_loader:
        y_pred = model(test_X.unsqueeze(1))
        loss = criterion(y_pred, test_y)
        total_loss += loss.item()
        y_true.extend(test_y.cpu().numpy())
        y_pred_list.extend(y_pred.cpu().numpy())

final_loss = total_loss / len(test_loader)
y_true = np.array(y_true)
y_pred = np.array(y_pred_list)
r2 = 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)
print(f'\nFinal R² Score: {r2:.3f}')


Final R² Score: 0.999
